# Modelado semántico
Autor: Eric S. Tellez <eric.tellez@infotec.mx>

El modelado léxico tiene importantes usos en la recuperación de información; los sistemas pueden ser altamente escalables si hay una implementación adecuada en índice invertido. Sin embargo, sufren de diversos problemas debido a su naturaleza:

- Las consultas deben ser expresadas con los mismos términos que los documentos.
- Los términos con múltiples significados pueden introducir falsos positivos en los resultados.
- En general se tienen problemas con la sinónimia, la homonimia, y la hiponimia, así como las variaciones léxicas resultantes de errores ortográficos, ya sean intencionados o involuntarios.

Aunque existen soluciones dos grandes vertientes de modelado semántico:
- Modelado del vocabulario: Intententa asignar cada palabra del vocabulario a un _concepto_ o significado _semántico_, que esta representado por un vector de alta dimensión (embedding).
- Modelado del lenguaje: Intenta modelar el lenguaje, normalmente aprendiendo que palabras pueden seguir a un contexto dado. La diferencia principal al modelado de vocabulario es que cuando se modela el lenguaje se tiene un modelo sensible al contexto, y por lo tanto, para una misma palabra, el contexto completo derminará su vector _embedding_. 

Aunque existen diversos intentos de modelado semántico en la literatura clásica, hasta tiempos recientes es cuando ha tomado gran interés debido a los impresionantes avances en redes neuronales profundas que han permitido aprender estos modelos de enormes colecciones, capturando una semántica profunda. Los grandes modelos de lenguaje han abierto nuevas posibilidades sobre las aplicaciones posibles en recuperación de información.

## Sobre las representaciones de documentos
Los modelos semánticos de palabras, anteriormente discutidos, son insuficientes para la representación de documentos; para conseguir representaciones de documentos hay diferentes estrategias. Por ejemplo, es posible generar combinaciones lineales de los embeddings individuales para generar vector embedding de documento. Algunos modelos basados en redes neuronales, generan embeddings de documento por diseño.


## Organización del resto de la unidad
Esta únidad esta dividida en dos notebooks. El primero (este documento) está dedicado al modelado semántico de vocabularios, mientras que el segundo se enfoca a modelos de lenguaje que trabajen sobre documentos. 

# Modelado semántico de vocabularios
El modelado basado en vocabularios o léxico tiene una gran cantidad de aplicaciones, las cuales son suelen explotar la gran cantidad de grados de libertad en el preprocesamiento, toquenizado y pesado, para ajustar a las tareas deseadas. Sin embargo, dicho modelado tiene problemas basados en la semántica:

- si las consultas tienen palabras no conocidas (fuera de vocabulario), no es posible ligar a palabras similares por semántica 
- las palabras similares en el vocabulario pero no iguales, no puede relacionarse para reducir un vocabulario, o para cualquier otro efecto
- las palabras que se escriben igual pero que tienen significado idéntico serán puestas en la misma bolsa

De manera más detallada, se tendrán problemas al manejar sinónimos, hipónimos, antónimos, etc. Adicionalmente, los errores serán tratados como palabras diferentes y no ajustados al mejor calce conocido (a menos que exista un corrector ortográfico entre el preprocesamiento).

Estos problemas se buscan reducir con representaciones semánticas de los vocabularios y de los textos, esto sería equivalente. El intento sería buscar por conceptos más que por palabras que lo describan.
En este punto aparecen los modelos semánticos, los cuales suelen ser representaciones vectoriales densas _embeddings_, de alta dimensión pero mucho menor que las representaciones léxicas (i.e., pasarían a unos pocos cientos en lugar de varias decenas de miles o cientos de miles de coordinadas).

# Word embeddings
Las representaciones de vectores densos de las palabras se suelen construir utilizando la [_hipótesis distribucional_](https://aclweb.org/aclwiki/Distributional_Hypothesis), que se puede resumir que las palabras con contextos similares (palabras que la rodean) tienen un significado similar. Entonces, para obtener la semántica, es necesario comparar contextos; lo anterior se puede llevar a cabo de diferentes maneras.

- Latent semantic analysis (LSA): La matriz $W_{m,n}^*$  con pesos basados en frecuencias es usada para reducir el numero de filas (palabras) mediante _Singular Value Decomposition_ (SVD). La similitud adecuada entre vectores en el espacio de dimensión reducida es el coseno. Ver [@Dumais2004] para más información.
- Latent semantic indexing (LSI): Similar a LSA pero usando esquemas de pesado diferentes, adicionalmente se usa Trucated SVD (TSVD) en lugar de SVD ya que puede seleccionar componentes más importantes. Para más detalles ver [@Hofman1999].
- Word2Vec. Se construye una red neuronal con una matriz de pesos del tamaño del vocabulario por una dimensión seleccionada, a partir de los contextos. Revisa los textos de entrenamiento mediante una ventana móvil. De manera especial, se tiene que el centro de la ventana es la palabra objetivo y el contexto esta a su alrededor. Se intenta predecir mediante el contexto la palabra central, y se ajusta mediante propagación hacia atrás. La función de perdida suele ser [softmax](https://en.wikipedia.org/wiki/Softmax_function) o hierachical softmax. Los embeddings no son la capa de salida sino una capa atrás. Para más detalles ver [@MCCD2013].
- Glove. Se utiliza una matriz de coocurrencia de términos dentro de una ventana deslizante que analiza una colección de textos. La matriz es altamente dispersa y de alta dimensión. Plantea un problema de optimización lineal para capturar la información más útil y bajar su dimensión. Los detalles se pueden consultar en [@PSM2014]. 
- FastText. Muy similar a Word2Vec pero añade manejo de palabras fuera de diccionario mediante subwords (similar a q-gramas de carácteres por cada palabra). Ver [@BGJM2017] para mayores detalles.

# Language models
En los word embeddings, cada palabra del vocabulario tiene asignado un vector denso de manera estática, fruto de su semántica basada en la hipótesis distribucional. Los modelos de lenguaje van más allá, intentando no solo tener en cuenta una palabra para el vector, si no que el vector mismo es dependiente del contexto, por lo cual puede desambiguar de manera natural palabras idénticas (homónimos) usando dicha información contextual. Adicionalmente, capturan información relevante de grandes corpus de texto, aportando muchas veces información de un mundo (i.e., si se entrenan usando la wikipedia, tendrán información relavante de multiples dominios).

La punta de lanza de los modelos de lenguaje es el aprendizaje profundo, y más precisamente, con el uso de _transformers_. Uno de los modelos de lenguaje más utilizado es BERT [@VSPU2017]. Estos modelos están fuera del alcance de nuestro curso pero se invitá al interesado a revisar más sobre ellos <https://huggingface.co/>.

## Nota
Este notebook esta basado en <https://github.com/sadit/SimilaritySearchDemos/blob/main/Glove/Glove.ipynb>. En el notebook original se muestran visualizaciones basadas en todos los vecinos cercanos (se verán más adelante en el curso).

# Ejemplo

El resto del documento muestra como usar word embeddings Glove para precalculados, para resolver tareas de vecinos cercanos y resolución de analogías.

In [1]:
using Pkg
Pkg.activate(".")
using SimilaritySearch, Plots, LinearAlgebra, Embeddings, HypertextLiteral

  Activating project at `~/Courses/IR-2024/Unidades`
Precompiling Embeddings
  ✓ Embeddings
  1 dependency successfully precompiled in 13 seconds. 25 already precompiled.


In [2]:
function load_dataset()
    emb = load_embeddings(GloVe, 2)  # you can change with any of the available embeddings in `Embeddings`
    for c in eachcol(emb.embeddings)
        normalize!(c)
    end

    db = MatrixDatabase(emb.embeddings)
    db, emb.vocab
end

function create_index()
    db, vocab = load_dataset()
    dist = NormalizedCosineDistance()
    index = SearchGraph(; dist, db, verbose=false)
    index!(index)
    optimize!(index, MinRecall(0.9))
    index, vocab
end


create_index (generic function with 1 method)

### El índice métrico se crea, además se toma el vocabulario

In [3]:
@time index, vocab = create_index()
vocab2id = Dict(w => i for (i, w) in enumerate(vocab));

 29.645612 seconds (7.35 M allocations: 3.046 GiB, 3.16% gc time, 6.07% compilation time: 2% of which was recompilation)


### Búsqueda de todos los vecinos cercanos en el vocabulario, observe la conveniencia del uso de un índice

In [ ]:
let k = 32
    @time iI, _ = allknn(index, k)
    # WARNING: Don't run the following line, it takes too much time
    @time gI, _ = allknn(ExhaustiveSearch(; db=index.db, dist=index.dist), k)
    macrorecall(gI, iI)
end

 65.035397 seconds (1.41 M allocations: 171.380 MiB, 0.97% compilation time)


### Búsqueda y presentación de los resultados

In [ ]:
function search_and_display(index, vocab, q, res, k, qword)
    res = reuse!(res, k)
    @time search(index, q, res)

    L = []
    for (j, p) in enumerate(res)
        push!(L, @htl "<tr><td>$j</td><td>$(vocab[p.id])</td><td>$p.id</td><td>$(round(p.weight, digits=3))</td></tr>")
    end

    display(@htl """<h2>resultados for "$qword"</h2>
    <table>
    <th>  <td>word</td> <td>id</td> <td>dist</td> </th>
        $L
    </table>
    """)
end

function search_and_display(index, vocab, qid::Integer, res, k=maxlength(res))
    search_and_display(index, vocab, index[qid], res, k, vocab[qid])
end

display(@htl "<h1>Ejemplos de búsqueda (aleatorios)</h1>")
res = KnnResult(7)
for i in 1:3
    for qid in rand(1:length(vocab))
        search_and_display(index, vocab, qid, res)
    end
end

In [ ]:
search_and_display(index, vocab, vocab2id["glove"], res, 15)

In [ ]:
# Analogias

In [ ]:
function analogy(a, b, c, k)
	v = index[vocab2id[a]] - index[vocab2id[b]] + index[vocab2id[c]]
	normalize!(v)
	search_and_display(index, vocab, v, res, k, "<$a> - <$b> + <$c>")
end

analogy("father", "man", "woman", 5)
analogy("fireman", "man", "woman", 5)
analogy("policeman", "man", "woman", 5)
analogy("mississippi", "usa", "france", 5)

# Actividades 
- Como se menciono, el uso de redes neuronales se vuelve muy popular para aprender la semántica desde el texto. Comente las razones justificando sus comentarios en términos de aprendizaje y requerimientos computacionales.
- Discuta que tipo de métrica se usa y porque.
- ¿Cómo se podrían usar los word-embeddings para búsqueda de documentos completos?
- Como ajustaría la métrica.
- Si usa Julia, revisé el paquete `Embeddings.jl`, `Word2Vec.jl`.
- Si usa Python, revisé los paquetes `fastText` y `word2vec`.
- Glove <https://nlp.stanford.edu/projects/glove/>
- Reproduzca el ejercicio de este notebook (no reproduzca la búsqueda exhaustiva, ya que le tomará muchísimo tiempo), use embeddings para español, cambié los ejemplos. Se sugiere el uso de <https://ingeotec.github.io/regional-spanish-models/> donde encontrará modelos fastText regionalizados del español, pero puede usar otros embeddings.
- Reporte su notebook y anote sus soluciones a las preguntas planteadas. Anote sus reflexiones.

# Bibliografía
- [Dumais2004] Dumais, S. T. (2004). Latent semantic analysis. Annu. Rev. Inf. Sci. Technol., 38(1), 188-230.
- [Hofmann1999] Hofmann, T. (1999, August). Probabilistic latent semantic indexing. In Proceedings of the 22nd annual international ACM SIGIR conference on Research and development in information retrieval (pp. 50-57).
- [MCCD2013]: Mikolov, T., Chen, K., Corrado, G., & Dean, J. (2013). Efficient estimation of word representations in vector space. arXiv preprint arXiv:1301.3781.
- [PSM2014] Pennington, J., Socher, R., & Manning, C. D. (2014, October). Glove: Global vectors for word representation. In Proceedings of the 2014 conference on empirical methods in natural language processing (EMNLP) (pp. 1532-1543).
- [BGJM2017]: Bojanowski, P., Grave, E., Joulin, A., & Mikolov, T. (2017). Enriching word vectors with subword information. Transactions of the association for computational linguistics, 5, 135-146.
- [VSPU2017] Vaswani, A., Shazeer, N., Parmar, N., Uszkoreit, J., Jones, L., Gomez, A. N., ... & Polosukhin, I. (2017). Attention is all you need. Advances in neural information processing systems, 30.